## Shellcode decryption

In [1]:
from Crypto.Cipher import ChaCha20

# Define the key and nonce
key = bytes.fromhex("943df638a81813e2de6318a507f9a0ba2dbb8a7ba63666d08d11a65ec914d66f")
nonce = bytes.fromhex("f236839f4dcd711a52862955")

with open("encrypted_shellcode.bin", "rb") as f:
    encrypted_data = f.read()
cipher = ChaCha20.new(key=key, nonce=nonce)
decrypted_data = cipher.decrypt(encrypted_data)

with open("decrypted_shellcode.bin", "wb") as f:
    f.write(decrypted_data)


## IP:PORT decoding (not useful too much)

In [2]:
## deocding IP and port
import struct
import socket

def ror(n, c, bits=64):
    mask = (1 << bits) - 1
    return ((n >> c) | (n << (bits - c))) & mask

def rol(n, c, bits=64):
    return ror(n, bits - c, bits)

def swap32(i):
    return struct.unpack("<I", struct.pack(">I", i))[0]

port = rol(1337,8,16)
packed_ip = struct.pack('!I',swap32(0xA00020F))
ip_address = socket.inet_ntoa(packed_ip)


print(f"IP: {ip_address}:{port}")
# print(hex(port))
# print(hex(swap32(0xA00020F)))


IP: 15.2.0.10:14597


## Custom chacha20 emulation

In [1]:
from unicorn import *
from unicorn.x86_const import *

key = bytes.fromhex("8dec9112eb760eda7c7d87a443271c35d9e0cb878993b4d904aef934fa2166d7")
nonce = bytes.fromhex("111111111111111111111111")
encrypted_flag = bytes.fromhex(
    "a9f63408422a9e1c0c03a8089470bb8daadc6d7b24ff7f247cda839e92f7071d0263902ec158"
)
ADDRESS = 0x1000000
# r8d -> counter
crypto_context_ptr = ADDRESS + 0x5000
key_ptr = ADDRESS + 0x5200
nonce_ptr = ADDRESS + 0x5300
encrypted_flag_ptr = ADDRESS + 0x5400

shellcode = None
with open("decrypted_shellcode.bin", "rb") as f:
    shellcode = f.read()

uc = Uc(UC_ARCH_X86, UC_MODE_64)
stack_base = 0x00010000
stack_size = 0x00010000
uc.mem_map(stack_base, stack_size)
uc.mem_write(stack_base, b"\x00" * stack_size)
uc.reg_write(UC_X86_REG_RSP, stack_base + stack_size // 2)
uc.reg_write(UC_X86_REG_RBP, stack_base + stack_size // 2)
uc.mem_map(ADDRESS, 0x00010000, UC_PROT_ALL)
uc.mem_write(ADDRESS, shellcode)

# seg000:0000000000000E99                 lea     rax, [rbp+crypto_context] ; crypto_context
# seg000:0000000000000EA0                 lea     rdx, [rbp+prob_key] ; prob_key
# seg000:0000000000000EA7                 lea     rcx, [rbp+prob_nonce] ; prob_nonce
# seg000:0000000000000EAE                 xor     r8d, r8d        ; prob_counter
# seg000:0000000000000EB1                 call    prob_chacha20_init

# write crypto_context address to RAX
uc.reg_write(UC_X86_REG_RAX, crypto_context_ptr)
# write key to ADDRESS+0x5100 and load address to RDX
uc.mem_write(key_ptr, key)
uc.reg_write(UC_X86_REG_RDX, key_ptr)
# write nonce to ADDRESS+0x5200 and load address to RCX
uc.mem_write(nonce_ptr, nonce)
uc.reg_write(UC_X86_REG_RCX, nonce_ptr)
uc.reg_write(UC_X86_REG_R8D, 0)

# emulation from ADDRESS+0x0CD2 to ADDRESS+0x0D42 (prob_chacha20_init from start to <leave> instruction)
uc.emu_start(ADDRESS + 0xCD2, ADDRESS + 0xD42, timeout=0, count=0)

## update reg state to enter the decryption routine
# seg000:0000000000000EB6                 lea     rax, [rbp+crypto_context] ; _BYTE * crypto_context
# seg000:0000000000000EBD                 lea     rdx, [rbp+file_content] ; _BYTE * file_content-> encypted_flag
# seg000:0000000000000EC4                 mov     ecx, [rbp+file_content_len] ; file_content_len
# seg000:0000000000000ECA                 call    decrypt

# reload RAX with a pointer to crypto context
uc.reg_write(UC_X86_REG_RAX, crypto_context_ptr)
# write encrypted flag data and load address to rdx and the size to ECX
uc.mem_write(encrypted_flag_ptr, encrypted_flag)
uc.reg_write(UC_X86_REG_RDX, encrypted_flag_ptr)
uc.reg_write(UC_X86_REG_ECX, len(encrypted_flag))

# emulation from ADDRESS+0x0D49 to ADDRESS+0x0D83 (decrypt function from start to <leave> instruction)
uc.emu_start(ADDRESS + 0xD49, ADDRESS + 0xD83, timeout=0, count=0)
print(f"Flag: {uc.mem_read(encrypted_flag_ptr,0x20).decode('utf-8')}")


Flag: supp1y_cha1n_sund4y@flare-on.com
